## Cat Dog Dataset

The training archive contains 25,000 images of dogs and cats. Train your algorithm on these files and predict the labels for test1.zip (1 = dog, 0 = cat).

You can obtain the dataset from https://www.kaggle.com/c/dogs-vs-cats/data

This case study is adopted from https://www.kaggle.com/uysimty/keras-cnn-dog-or-cat-classification

### 1. Import Library

In [1]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from _data_new import organise_images_in_folders
import os, logging, glob
from _logging import set_logging
from _utility import get_perc

set_logging(logging)

organise_images_flag = False
data_folder = "cats_dogs_small"
classes = ["cat", "dog"]
dict_classes = {"cat": 0, "dog": 1}
if organise_images_flag:
    organise_images_in_folders(data_folder, classes)

#class of gpu
class gpu:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

# class of hyperparameters
class hyper:
    lr = 0.01
    batch_size = 100
    epochs = 10
    


d:\Anaconda\envs\fastai\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Define Constants

In [2]:
FAST_RUN = True #False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

### 3. Prepare Traning Data

In [3]:
def get_data(data_folder, type):
    file_path = os.path.join(data_folder, "data", type, "*\\*")
    list = glob.glob(file_path)
    X_list = random.sample(list, len(list))
    labels = []
    for item in X_list:
        temp = item.split(".")
        labels.append(dict_classes["cat"] if temp[0].endswith("cat") else dict_classes["dog"])
        
    return X_list, labels
        
        

In [4]:
X_train, y_train = get_data(data_folder, "train")
X_test, y_test = get_data(data_folder, "test")
dogs_perc_train = get_perc(sum(y_train), len(y_train))
dogs_perc_test = get_perc(sum(y_test), len(y_test))
logging.info(f"{dogs_perc_train}\% of the training images are labelled as dogs")
logging.info(f"{dogs_perc_test}\% of the test images labelled as dogs")
# get labels

2023-01-20 20:13:21,988 | INFO : 50.24\% of the training images are labelled as dogs
2023-01-20 20:13:21,990 | INFO : 50.24\% of the test images labelled as dogs


#### See Total In count

In [6]:
number_dogs = sum(y_train) + sum(y_test)
number_cats = len(y_train) + len(y_test) - number_dogs
logging.info(f"From our data we have {number_cats} cats and {number_dogs} dogs")

2023-01-20 20:20:10,280 | INFO : From our data we have 1264 cats and 1276 dogs


#### See sample image